In [ ]:
import numpy as np
import pandas as pd
import datetime
from PriceFetcher import PriceFetcher
from MarketCorrections import MarketCorrections
from Utilities import period_max_drawdown

In [ ]:
pf = PriceFetcher(assets=["SPY", "AGG"])
pf.fetch()
df_px = pf.prices

[*********************100%***********************]  2 of 2 completed


In [ ]:
mc = MarketCorrections(asset="SPY", correction=-0.05)
df_corrections = mc.corrections

[*********************100%***********************]  1 of 1 completed


In [ ]:
portfolio = {
    "spy":0.5,
    "agg":0.5
}
date_start = datetime.date(2007, 4, 11)
date_end = datetime.date(2024, 12, 31)

In [ ]:
class DailyRebalanceBacktester:
    """
    A backtester for a weighted portfolio of assets with daily rebalancing of the weights.

    Attributes
    ----------
    portfolio: dict[str, float]
        Defines the assets and weights in the portfolio.
        
    prices: pd.DataFrame
        Contains the prices of assets as far back as there is data for in Yahoo Finance.
        This is typically the result of the PriceFetcher.fetch() method.
        
    market_corrections: pd.DataFrame
        Contains the start, bottom, end date of prices corrections of a particular asset.
        Typically some kind of broad market index like SPY will be used.  This is usually
        the result of the MarketCorrections class.  It is modified by the calc_period_drawdown
        method to contain the drawdowns of the weighted portfolio during the drawdown periods.
        
    date_start: datetime.date
        The start date of the backtest.
        
    date_end: datetime.date
        The end date of the backtest.

    assets: list[str]
        The component assets in the portfolio of the backtest.  This is extracted from the portfolio.
    
    weights: list[str]
        The weights of the component assets in the portfolio.  This is extracted from the portfolio

    returns: pd.DataFrame
        The prices, daily returns, equity curve, drawdowns of the assets
        and the weighted portfolio that is being backtested.

    cumulative_returns: dict[str, float]
        The cumulative returns for each of the component assets and the weighted portfolio.

    annual_returns: dict[str, float]
        The annualized returns for each of the component assets and the weighted portfolio.

    volatility: dict[str, float]
        The annualized volatility for each of the component assets and the weighted portfolio.

    sharpe_ratio: dict[str, float]
        The annualized sharpe-ratio for each of the component assets and the weighted portfolio.

    sharpe_ratio: dict[str, float]
        The annualized sharpe-ratio for each of the component assets and the weighted portfolio.

    drawdown_max: dict[str, float]
        The maximum for each of the component assets and the weighted portfolio.

    annual_performance: pd.DataFrame
        The performance of the weighted portfolio for each calendar year in the backtest.
    """
    def __init__(self, 
                 portfolio: dict[str, float], 
                 prices: pd.DataFrame,
                 market_corrections: pd.DataFrame,
                 date_start: datetime.date,
                 date_end: datetime.date):
        self.portfolio = portfolio
        self.prices = prices        
        self.market_corrections = (
            market_corrections
                .query("@date_start <= start & end <= @date_end").copy()
        )
        """
        portfolio: dict[str, float]
            Defines the assets and weights in the portfolio.
            
        prices: pd.DataFrame
            Contains the prices of assets as far back as there is data for in Yahoo Finance.
            This is typically the result of the PriceFetcher.fetch() method.
            
        market_corrections: pd.DataFrame
            Contains the start, bottom, end date of prices corrections of a particular asset.
            Typically some kind of broad market index like SPY will be used.  This is usually
            the result of the MarketCorrections class.  It is modified by the calc_period_drawdown
            method to contain the drawdowns of the weighted portfolio during the drawdown periods.
            
        date_start: datetime.date
            The start date of the backtest.
            
        date_end: datetime.date
            The end date of the backtest.
        """
        
        self.date_start = date_start
        self.date_end = date_end

        # isolating weights and assets from portfolio
        self.assets = []
        self.weights = []
        for asset, weight in self.portfolio.items():
            self.assets.append(asset)
            self.weights.append(weight)

    
    def calc_daily_returns(self) -> None:
        """
        Calculates the prices, daily returns, equity curve, drawdowns of the assets
        and the weighted portfolio that is being backtested.
        """
        
        self.returns = self.prices.query("@date_start <= date & date <= @date_end").copy()

        # calculating component asset daily returns
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            self.returns[ret_col_name] = self.returns[ix_asset].pct_change()
        self.returns.fillna(0, inplace=True)

        # calculating portfolio daily returns
        cols = []
        for ix_asset in self.assets:
            cols.append("ret_" + ix_asset)
        self.returns["ret_portfolio"] =  np.sum(np.array(self.returns[cols]) * self.weights, axis=1)

        # calculating equity curve for components and portfolio
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            equity_col_name = "equity_" + ix_asset
            self.returns[equity_col_name] = (1 + self.returns[ret_col_name]).cumprod()
        self.returns["equity_portfolio"] = (1 + self.returns["ret_portfolio"]).cumprod()

        # calculating drawdowns for components and portfolio
        for ix_asset in self.assets:
            equity_col_name = "equity_" + ix_asset
            drawdown_col_name = "drawdown_" + ix_asset
            self.returns[drawdown_col_name] = (self.returns[equity_col_name] / self.returns[equity_col_name].cummax()) - 1
        self.returns["drawdown_portfolio"] = (self.returns["equity_portfolio"] / self.returns["equity_portfolio"].cummax()) - 1

        
    def calc_portfolio_statistics(self) -> None:
        """
        Calculates the portfolio statistics and annual performance of the component assets
        and the weighted portfolio being backtested.
        """
        # cumulative return
        self.cumulative_return = {}
        for ix_asset in self.assets:
            equity_col_name = "equity_" + ix_asset
            self.cumulative_return[ix_asset] = (self.returns[equity_col_name].iloc[-1] - 1)
        self.cumulative_return["portfolio"] = self.returns["equity_portfolio"].iloc[-1] - 1
        
        # annual return
        self.annual_return = {}
        for ix_asset in self.assets:
            equity_col_name = "equity_" + ix_asset
            self.annual_return[ix_asset] = (self.returns[equity_col_name].iloc[-1] ** (252/(len(self.returns) - 1)) - 1)
        self.annual_return["portfolio"] = self.returns["equity_portfolio"].iloc[-1] ** (252/(len(self.returns) - 1)) - 1

        # volatility
        self.volatility = {}
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            self.volatility[ix_asset] = self.returns[ret_col_name][1:].std() * np.sqrt(252)
        self.volatility["portfolio"] = self.returns["ret_portfolio"][1:].std() * np.sqrt(252)

        # sharpe-ratio
        self.sharpe_ratio = {}
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            self.sharpe_ratio[ix_asset] = (self.returns[ret_col_name][1:].mean() / self.returns[ret_col_name][1:].std()) * np.sqrt(252)
        self.sharpe_ratio["portfolio"] = (self.returns["ret_portfolio"][1:].mean() / self.returns["ret_portfolio"][1:].std()) * np.sqrt(252)
        
        # maximum drawdown
        self.drawdown_max = {}
        for ix_asset in self.assets:
            drawdown_col_name = "drawdown_" + ix_asset
            self.drawdown_max[ix_asset] = self.returns[drawdown_col_name].min()
        self.drawdown_max["portfolio"] =  self.returns["drawdown_portfolio"].min()
        
        # annual performance
        df_portfolio = self.returns[["date", "ret_portfolio"]].copy()
        df_portfolio["date"] = pd.to_datetime(df_portfolio["date"])
        df_portfolio["year"] = df_portfolio["date"].dt.year
        self.annual_performance = df_portfolio.groupby(["year"])[["ret_portfolio"]].agg(lambda x: np.prod(1 + x) - 1).reset_index()

    def calc_period_drawdowns(self) -> None:
        """
        Calculates the performance of the weighted portfolio during the drawdown periods.
        """
        drawdowns_portfolio = []
        for ix in self.market_corrections.index:
            dt_start = self.market_corrections.at[ix, "start"]
            dt_end = self.market_corrections.at[ix, "end"]
            drawdown_portfolio = period_max_drawdown(asset="portfolio", date_start=dt_start, date_end=dt_end, df_ret=self.returns)
            drawdowns_portfolio.append(drawdown_portfolio)
        self.market_corrections["drawdown_portfolio"] = drawdowns_portfolio


In [ ]:
drb = DailyRebalanceBacktester(portfolio, df_px, df_corrections, date_start, date_end)
drb.calc_daily_returns()
drb.calc_portfolio_statistics()
drb.calc_period_drawdowns()

In [ ]:
print(drb.portfolio)
print(drb.assets)
print(drb.weights)

{'spy': 0.5, 'agg': 0.5}
['spy', 'agg']
[0.5, 0.5]


In [ ]:
drb.prices.head()

,date,agg,spy
0,1993-01-29,NaN,24.526079
1,1993-02-01,NaN,24.700510
2,1993-02-02,NaN,24.752850
3,1993-02-03,NaN,25.014496
4,1993-02-04,NaN,25.119156


In [ ]:
drb.market_corrections.head()

,start,end,bottom,drawdown_spy,drawdown_portfolio
15,2007-07-19,2007-10-05,2007-08-15,-0.090475,-0.040855
16,2007-10-09,2012-08-16,2009-03-09,-0.551895,-0.286754
17,2012-09-14,2013-01-02,2012-11-15,-0.073456,-0.034661
18,2013-05-21,2013-07-11,2013-06-24,-0.055506,-0.045349
19,2013-12-31,2014-02-24,2014-02-03,-0.056960,-0.022383


In [ ]:
drb.returns

,date,agg,spy,ret_spy,ret_agg,ret_portfolio,equity_spy,equity_agg,equity_portfolio,drawdown_spy,drawdown_agg,drawdown_portfolio
3575,2007-04-11,58.578686,102.853485,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
3576,2007-04-12,58.649265,103.310608,0.004444,0.001205,0.002825,1.004444,1.001205,1.002825,0.000000,0.000000,0.000000
3577,2007-04-13,58.555176,103.781952,0.004562,-0.001604,0.001479,1.009027,0.999599,1.004308,0.000000,-0.001604,0.000000
3578,2007-04-16,58.672722,104.767471,0.009496,0.002007,0.005752,1.018609,1.001605,1.010084,0.000000,0.000000,0.000000
3579,2007-04-17,58.843208,105.045998,0.002659,0.002906,0.002782,1.021317,1.004516,1.012895,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
8033,2024-12-24,96.450081,601.299988,0.011115,0.001138,0.006126,5.846180,1.646505,3.411381,-0.007381,-0.090591,-0.014916
8034,2024-12-26,96.519844,601.340027,0.000067,0.000723,0.000395,5.846569,1.647696,3.412729,-0.007315,-0.089933,-0.014527
8035,2024-12-27,96.320511,595.010010,-0.010527,-0.002065,-0.006296,5.785025,1.644293,3.391243,-0.017764,-0.091813,-0.020731
8036,2024-12-30,96.699249,588.219971,-0.011412,0.003932,-0.003740,5.719009,1.650758,3.378560,-0.028973,-0.088242,-0.024393


In [ ]:
print(drb.cumulative_return)
print(drb.annual_return)
print(drb.volatility)
print(drb.sharpe_ratio)
print(drb.drawdown_max)

{'spy': 4.698202802537333, 'agg': 0.6487165873570029, 'portfolio': 2.3703251937481693}
{'spy': 0.10326984019858254, 'agg': 0.028640777910573956, 'portfolio': 0.07102911374866294}
{'spy': 0.19866949846077295, 'agg': 0.054752781281243196, 'portfolio': 0.10271373446052512}
{'spy': 0.5942314641379826, 'agg': 0.543267927232031, 'portfolio': 0.71948068933058}
{'spy': -0.5518946144758285, 'agg': -0.18432916212120753, 'portfolio': -0.28675442344797664}


In [ ]:
drb.annual_performance

,year,ret_portfolio
0,2007,0.045415
1,2008,-0.154988
2,2009,0.151749
3,2010,0.111494
4,2011,0.055851
5,2012,0.099720
6,2013,0.140679
7,2014,0.098790
8,2015,0.012120
9,2016,0.073573


In [ ]:
drb.market_corrections

,start,end,bottom,drawdown_spy,drawdown_portfolio
15,2007-07-19,2007-10-05,2007-08-15,-0.090475,-0.040855
16,2007-10-09,2012-08-16,2009-03-09,-0.551895,-0.286754
17,2012-09-14,2013-01-02,2012-11-15,-0.073456,-0.034661
18,2013-05-21,2013-07-11,2013-06-24,-0.055506,-0.045349
19,2013-12-31,2014-02-24,2014-02-03,-0.056960,-0.022383
20,2014-09-18,2014-10-31,2014-10-16,-0.072734,-0.026677
21,2015-07-20,2016-04-18,2016-02-11,-0.130229,-0.057273
22,2016-06-08,2016-07-08,2016-06-27,-0.055243,-0.023143
23,2018-01-26,2018-08-06,2018-02-08,-0.101019,-0.056721
24,2018-09-20,2019-04-12,2018-12-24,-0.193489,-0.093241


## OLD CODE

In [ ]:
# def get_market_data(self):
#     # getting asset prices
#     pf = PriceFetcher(assets=[x.upper() for x in self.assets])
#     pf.fetch()
#     self.prices = pf.prices
#     pf = None

#     # getting market corrections
#     mc = MarketCorrections(
#         asset=self.reference_market.upper(),
#         correction=self.correction_size,
#     )
#     self.market_corrections = \
#         mc.corrections.query("@self.date_start <= start & end <= @self.date_end")